In [1]:
import sympy as smp
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
class FileSystemManager:
   def __init__(self):
        pass
   def open_file(self, text, filepath, flag):
        """
        Opens a file in read or write mode.
        
        :param text: Text to write to the file if flag is 'w'
        :param filepath: Path of the file to open
        :param flag: 'r' to read, 'w' to write
        :return: File content if reading, None if writing
        """
        try:
            if flag == 'r':
                with open(filepath, 'r') as f:
                    data = f.read()
                    return data  # return instead of print
            elif flag == 'w':
                with open(filepath, 'w') as f:
                    f.write(text)
            else:
                raise ValueError("Unsupported flag. Use 'r' for read or 'w' for write.")
        except FileNotFoundError:
            print(f"Error: File {filepath} not found.")
        except IOError as e:
            print(f"Error accessing file {filepath}: {e}")

In [3]:
class MagmaCalculator:
    """
    A class to interact with the Magma Calculator webpage and submit code for evaluation.
    
    Attributes:
    ----------
    output_file : str
        The name of the file where the output from the Magma Calculator will be saved.
    driver : webdriver.Chrome
        A Chrome web driver instance to automate browser interaction.
    url : str
        The URL of the Magma Calculator page.
    
    Methods:
    -------
    submit_code(code):
        Submits the given Magma code to the calculator and saves the result to a file.
    
    close():
        Closes the browser session.
    """
    
    def __init__(self, output_file="MagmaCalcResult"):
        """
        Initializes the MagmaCalculator with the given output file name.
        
        Parameters:
        ----------
        output_file : str, optional
            The name of the file where the result will be saved (default is "output.txt").
        """
        self.url = "http://magma.maths.usyd.edu.au/calc/"
        self.output_file = output_file
        self.file_manager = FileSystemManager()

    def submit_code(self, code):
        """
        Submits the given code to the Magma Calculator and saves the result to the output file.
        
        Parameters:
        ----------
        code : str
            The Magma code to be submitted for evaluation.
        
        Actions:
        -------
        - Opens the Magma Calculator webpage.
        - Finds the input box and enters the code.
        - Clicks the submit button.
        - Waits for the result to load.
        - Retrieves the result and writes it to the specified output file.
        """
        driver = webdriver.Chrome()
        try:
            driver.get(self.url)
            
            input_box = driver.find_element(By.ID, "input")
            input_box.clear()
            input_box.send_keys(code)
            
            submit_button = driver.find_element(By.XPATH, "//input[@value='Submit']")
            submit_button.click()
            
            time.sleep(5)
            
            result_element = driver.find_element(By.ID, "result")
            result_text = result_element.get_attribute('value')
            
            self.file_manager.open_file(result_text, self.output_file, 'w')
        finally:
            driver.quit()
        


In [4]:
class FlynnLeprevost:
    def __init__(self, m=11, g=2, deg=5, curves_path="foundCurves.txt", magma_result_path="MagmaCalcResult"):
        """"""
        self.x, self.y = smp.symbols('x y')
        self.a1, self.b1, self.a2, self.b2 = smp.symbols('a1 b1 a2 b2')
        self.V1, self.V2, self.W1, self.W2 = smp.symbols('V1 W1 V2 W2') 
        self.m = m
        self.g = g
        self.deg = deg
        self.constraints = [self.deg, self.deg + self.g]
        self.coeffs = []
        self.curves_path = curves_path
        self.magma_calc = MagmaCalculator(magma_result_path)
        self.file_manager = FileSystemManager()

    def factor_coeffs(self, f, val):
        """
        Factors the coefficients of a polynomial and computes the sum of 
        each factored coefficient multiplied by a power of the given value.
        
        :param f: The polynomial expression whose coefficients will be factored.
        :param val: The symbol or value to use as the base for powers in the sum.
        
        :return: The sum of the factored coefficients of the polynomial `f`,
                each multiplied by the corresponding power of `val`.
                
        Example:
        If f = 2*x**2 + 3*x + 4 and val = x, the function will factor each 
        coefficient (if possible) and return the sum 4 + 3*x + 2*x**2.
        """
        return sum(smp.factor(coeff)*val**i for i, coeff in enumerate(reversed(smp.Poly(f, val).all_coeffs())))
    
    def calculate_torsion_point_order_divisor(self, a1, b1, a2, b2):
        """Calculate the divisor of torsion point order using the determinant."""
        matrix = smp.Matrix([[a2, b2], [a1, b1]])
        m = abs(matrix.det())
        return m
    
    def check_order(self, equation):
        """Function that helps to check an order of possible torison points of given function"""
        code = f"""
        P<x> := PolynomialRing(Rationals());
        f := {equation};
        f := PolynomialRing(Integers())!f;
        C1 := HyperellipticCurve(f);
        J1 := Jacobian(C1);
        TorsionSubgroup(J1);
        """
        self.magma_calc.submit_code(code, self.magma_calc.output_file)

    def divisor_deg(self, V, W):
        """
        Checks the divisor degree of given function
        deg*: degree of divisor
        deg: degree  of polynomial 
        deg* u_j = max(2 deg(V_j), 2 deg(W_j) + 2g + 1)
        """
        return max(2* smp.degree(V), 2 * smp.degree(W) + 2 * self.g + 1)

    def in_constraints(self, coeffs_set):
        return (coeffs_set[0] + coeffs_set[1] <= self.constraints[1]) and (coeffs_set[0] + coeffs_set[1] >= self.constraints[0]) and (coeffs_set[2] + coeffs_set[3] <= self.constraints[1]) and (coeffs_set[2] + coeffs_set[3] >= self.constraints[0])

    def get_coeffs(self, range_limit=11):
        values = [i for i in range(1, range_limit)]
        final_coeffs = []
        coeffs_sets = [[a1, b1, a2, b2] for a1 in values for b1 in values for a2 in values for b2 in values]
        for coeffs_set in coeffs_sets:
            if self.calculate_torsion_point_order_divisor(*coeffs_set) == self.m and self.in_constraints(coeffs_set):
                final_coeffs.append(coeffs_set)
        return final_coeffs
    
    def get_systems(self):
        coeffs_sets = self.get_coeffs(range_limit=11)
        systems = []
        for coeffs_set in coeffs_sets:
            tilda_deg_y = self.deg
            max_deg_V1 = int(np.floor((coeffs_set[2] + coeffs_set[3]) / 2))
            max_deg_V2 = int(np.floor((coeffs_set[0] + coeffs_set[1]) / 2))
            max_deg_W1 = int(np.floor((coeffs_set[2] + coeffs_set[3] - tilda_deg_y) / 2))
            max_deg_W2 = int(np.floor((coeffs_set[0] + coeffs_set[1] - tilda_deg_y) / 2))

            V1 = sum(smp.symbols(f'c1_{i}') * self.x**i for i in range(max_deg_V1 + 1))
            V2 = sum(smp.symbols(f'c2_{i}') * self.x**i for i in range(max_deg_V2 + 1))
            W1 = sum(smp.symbols(f'd1_{i}') * self.x**i for i in range(max_deg_W1 + 1))
            W2 = sum(smp.symbols(f'd2_{i}') * self.x**i for i in range(max_deg_W2 + 1))

            u1 = V1 + W1 * self.y
            u2 = V2 + W2 * self.y

            systems.append({
                'coeffs_set': coeffs_set,
                'V1': V1,
                'V2': V2,
                'W1': W1,
                'W2': W2,
                'u1': u1,
                'u2': u2
            })

        return systems
    
    def involution(self, equation):
        x, y = smp.symbols('x y')
        return equation.subs(y, -y)

    def get_final_systems(self):
        final_systems = []
        systems = self.get_systems()
        
        for system in systems:
            u1 = system['u1']
            u2 = system['u2']
            coeffs_set = system['coeffs_set']

            left_part_1 = smp.simplify(u1 * self.involution(u1))
            left_part_2 = smp.simplify(u2 * self.involution(u2))
            
             
            gamma1 = smp.symbols('gamma1')  
            power_x = smp.symbols('n')     
            power_x_minus_1 = smp.symbols('m')  

            equation = gamma1 * self.x**power_x * (self.x - 1)**power_x_minus_1 

            right_part_1 = equation.subs({power_x: coeffs_set[2], power_x_minus_1: coeffs_set[3]})
            right_part_2 = equation.subs({power_x: coeffs_set[0], power_x_minus_1: coeffs_set[1]})
            
            system_eqs = {
                'coeffs_set': system['coeffs_set'],
                'left_part_1': left_part_1,
                'left_part_2': left_part_2,
                'right_part_1': right_part_1,
                'right_part_2': right_part_2,
                'final_system': f"{left_part_1} = {right_part_1} and {left_part_2} = {right_part_2}"

            }
            
            final_systems.append(system_eqs)
    
        return final_systems

In [5]:
model = FlynnLeprevost()

In [6]:
model.get_final_systems()

[{'coeffs_set': [1, 5, 3, 4],
  'left_part_1': (c1_0 + c1_1*x + c1_2*x**2 + c1_3*x**3 - y*(d1_0 + d1_1*x))*(c1_0 + c1_1*x + c1_2*x**2 + c1_3*x**3 + y*(d1_0 + d1_1*x)),
  'left_part_2': (c2_0 + c2_1*x + c2_2*x**2 + c2_3*x**3 - d2_0*y)*(c2_0 + c2_1*x + c2_2*x**2 + c2_3*x**3 + d2_0*y),
  'right_part_1': gamma1*x**3*(x - 1)**4,
  'right_part_2': gamma1*x*(x - 1)**5,
  'final_system': '(c1_0 + c1_1*x + c1_2*x**2 + c1_3*x**3 - y*(d1_0 + d1_1*x))*(c1_0 + c1_1*x + c1_2*x**2 + c1_3*x**3 + y*(d1_0 + d1_1*x)) = gamma1*x**3*(x - 1)**4 and (c2_0 + c2_1*x + c2_2*x**2 + c2_3*x**3 - d2_0*y)*(c2_0 + c2_1*x + c2_2*x**2 + c2_3*x**3 + d2_0*y) = gamma1*x*(x - 1)**5'},
 {'coeffs_set': [2, 3, 5, 2],
  'left_part_1': (c1_0 + c1_1*x + c1_2*x**2 + c1_3*x**3 - y*(d1_0 + d1_1*x))*(c1_0 + c1_1*x + c1_2*x**2 + c1_3*x**3 + y*(d1_0 + d1_1*x)),
  'left_part_2': (c2_0 + c2_1*x + c2_2*x**2 - d2_0*y)*(c2_0 + c2_1*x + c2_2*x**2 + d2_0*y),
  'right_part_1': gamma1*x**5*(x - 1)**2,
  'right_part_2': gamma1*x**2*(x - 1)**3,


In [77]:
code =[ """
    Z := Integers();
    p := Factorial(10);
    p;
""",
"""
    P<x> := PolynomialRing(Rationals());
    f := 15876 * 21 * 49 * 7 * (50*x^5 - 1289375/15876*x^4 - 2075/21*x^3 + 13800/49*x^2 - 1350/7*x + 2025/49);
    f := PolynomialRing(Integers())!f;
    C1 := HyperellipticCurve(f);
    J1 := Jacobian(C1);
    TorsionSubgroup(J1);
    """
    ]
# Initialize the MagmaCalculator class
magma_calc = MagmaCalculator()
magma_calc.submit_code(code[1])
